In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
import time 
from pandas import Series
%matplotlib inline

# 定义文件全局变量
path = "/Users/skyamz/Desktop/JDD_Loan_Forecasting_Qualification/initdata/"
generate_path = "/Users/skyamz/Desktop/JDDb_Loan_Forecasting_Qualification/generate_data/"


# 加载文件
file_order = pd.read_csv(path + "t_order.csv")
file_click = pd.read_csv(path + "t_click.csv")
file_user = pd.read_csv(path + "t_user.csv")
file_loan = pd.read_csv(path + "t_loan.csv")
file_loan_sum = pd.read_csv(path + "t_loan_sum.csv")

# 格式化时间
file_order['buy_time_format'] = pd.to_datetime(file_order['buy_time'])
file_loan['loan_time_format'] = pd.to_datetime(file_loan['loan_time'])

# 创建索引
file_order = file_order.set_index('buy_time_format')
file_loan = file_loan.set_index('loan_time_format')

#聚合用户订单月份数据
def user_order_month_aggregate(month):
    # 取月份数据
    file_order_month = file_order[month]
    # 重置索引
    file_order_month = file_order_month.reset_index()
    # 统计用户订单某个类目的价格、数量、折扣
    file_order_month_sum = file_order_month.groupby(['uid','cate_id'])['price','qty','discount'].sum()
    # 重置索引
    file_order_month_sum = file_order_month_sum.reset_index()
    # 合并2列
    file_order_month_sum['cate_id_and_count'] = file_order_month_sum['cate_id'].astype('str') + ':' +  file_order_month_sum['qty'].astype('str')
    # 统计用户下单类目
    file_order_month_sum_unique_cate_list = file_order_month_sum.groupby('uid')['cate_id_and_count'].unique()
    # 重置索引
    file_order_month_sum_unique_cate_list = file_order_month_sum_unique_cate_list.reset_index()
    # 统计用户订单全部价格、数量、折扣
    file_order_month_total = file_order_month.groupby(['uid'])['price','qty','discount'].sum()
    # 重置索引
    file_order_month_total = file_order_month_total.reset_index()
    # 将数据进行聚合操作
    user_order_month_aggregate = pd.merge(file_order_month_sum_unique_cate_list,file_order_month_total,how='outer',on='uid')
    return user_order_month_aggregate

# 索引各个月份数据
file_order_month_aggregate_11 = user_order_month_aggregate('2016-11')
file_order_month_aggregate_10 = user_order_month_aggregate('2016-10')
file_order_month_aggregate_09 = user_order_month_aggregate('2016-09')
file_order_month_aggregate_08 = user_order_month_aggregate('2016-08')

# 合并
file_click['pid_param'] = file_click['pid'].astype('str') +'_' + file_click['param'].astype('str')

# 用户点击统计
user_click_count = pd.DataFrame(file_click.groupby(['uid','pid_param'])['uid'].count())

# 重命名统计列
user_click_count.columns = ['click_count']

# 重置索引,并设置索引为uid
user_click_count_detail = user_click_count.reset_index().set_index('uid')

# 统计用户的总点击次数
user_click_count_total = pd.DataFrame(user_click_count.groupby('uid')['click_count'].sum())

# 重命名列名称
user_click_count_total.columns = ['click_count_total']

# 重置索引
user_click_count_total = user_click_count_total.reset_index()

# 合并2列为字典
user_click_count_detail['param_and_click'] = user_click_count_detail['pid_param'].astype('str') + ':' + user_click_count_detail['click_count'].astype('str')

# 合并分组
user_click_count_detail_aggregate = pd.DataFrame(user_click_count_detail.groupby('uid')['param_and_click'].unique())

# 重置索引
user_click_count_detail_aggregate = user_click_count_detail_aggregate.reset_index()

# 用户点击聚合
user_click_count_aggregate = pd.merge(user_click_count_detail_aggregate,user_click_count_total,how='outer',on='uid')

# 各月用户点击和订单数据汇总
user_click_count_and_order_month_aggregate_08 = pd.merge(user_click_count_detail_aggregate,file_order_month_aggregate_08,how='outer',on='uid')
user_click_count_and_order_month_aggregate_09 = pd.merge(user_click_count_detail_aggregate,file_order_month_aggregate_09,how='outer',on='uid')
user_click_count_and_order_month_aggregate_10 = pd.merge(user_click_count_detail_aggregate,file_order_month_aggregate_10,how='outer',on='uid')
#user_click_count_and_order_month_aggregate_11 = pd.merge(user_click_count_detail_aggregate,file_order_month_aggregate_11,how='outer',on='uid')

# 重置索引
user_click_count_detail = user_click_count_detail.reset_index()

# 统计用户最大点击
user_click_count_detail_max = user_click_count_detail.loc[user_click_count_detail.groupby(['uid'])['click_count'].idxmax()].sort_values(by='uid')

# 查询每月订单最大数量
def file_order_month_max(month):
    file_order_month_sum = file_order[month].reset_index().groupby(['uid','cate_id'])['price','qty','discount'].sum().reset_index()
    file_order_month_sum.loc[file_order_month_sum.groupby(['uid'])['qty'].idxmax()].sort_values(by='uid')
    file_order_month_sum_max = file_order_month_sum.loc[file_order_month_sum.groupby(['uid'])['qty'].idxmax()].sort_values(by='uid')
    file_order_month_sum_max.columns = ['uid','cate_id_by_max_qty','price_by_max_qty','qty_max','discount_by_max_qty']
    return file_order_month_sum_max

# 查询点击最多次数和购买数量最大
def click_count_max_and_order_qty_max(month):
    click_count_max_and_order_qty_max = pd.merge(user_click_count_detail_max,file_order_month_max(month),how='outer',on='uid')
    click_count_max_and_order_qty_max = pd.merge(click_count_max_and_order_qty_max,file_user,how='outer',on='uid')
    return click_count_max_and_order_qty_max

# 统计贷款笔数
def loan_count(month):
    file_loan_count = pd.DataFrame(file_loan[month].groupby('uid')['uid'].count())
    file_loan_count.columns = ['loan_count']
    file_loan_count = file_loan_count.reset_index()
    file_loan_sum = pd.DataFrame(file_loan[month].groupby(['uid'])['loan_amount'].sum())
    file_loan_sum.columns = ['loan_sum']
    file_loan_sum = file_loan_sum.reset_index()
    file_loan_count_and_sum = pd.merge(file_loan_count,file_loan_sum,how='outer',on='uid')
    file_loan_count_and_sum['loan_sum_mean'] = file_loan_count_and_sum['loan_sum']/file_loan_count_and_sum['loan_count']
    return file_loan_count_and_sum

#打标用户当月是否借贷
def is_month_loan_user(month):
    loan_month = file_loan[month]
    is_loan_user = loan_month['uid'].unique()
    is_loan_user = pd.DataFrame(Series(is_loan_user))
    is_loan_user.columns = ['uid']
    is_loan_user['is_loan'] = 1
    return is_loan_user

#构建每个月的用户特征
def feature_month_uid(month,prefix_name):
    click_count_max_and_order_qty_max_tmp = click_count_max_and_order_qty_max(month)
    loan_count_tmp = loan_count(month)
    feature_month_uid = pd.merge(click_count_max_and_order_qty_max_tmp,loan_count_tmp,how='outer',on='uid')
    is_month_loan_user_tmp = is_month_loan_user(month)
    feature_month_uid = pd.merge(feature_month_uid,is_month_loan_user_tmp,how='outer',on='uid')
    feature_month_uid['diff'] = feature_month_uid['limit'] - feature_month_uid['loan_sum_mean']
    #one_hot_pid_param = pd.get_dummies(feature_month_uid['pid_param'])
    #one_hot_cate_id_by_max_qty = pd.get_dummies(feature_month_uid['cate_id_by_max_qty'])
    #one_hot_pid_param['uid'] = feature_month_uid['uid']
    #one_hot_cate_id_by_max_qty['uid'] = feature_month_uid['uid']
    #one_hot_all = pd.merge(one_hot_pid_param,one_hot_cate_id_by_max_qty,how = 'outer',on='uid')
    #feature_month_uid = pd.merge(feature_month_uid,one_hot_all,how='outer',on='uid')
    del feature_month_uid['active_date']
    del feature_month_uid['param_and_click']
    del feature_month_uid['pid_param']
    del feature_month_uid['cate_id_by_max_qty']
    del feature_month_uid['sex']
    feature_month_uid['age']
    feature_month_uid['limit']
    feature_month_uid.columns = [col+prefix_name for col in feature_month_uid.columns.values.astype('str')]
    feature_month_uid = feature_month_uid.rename(columns={"uid"+prefix_name:"uid"})
    feature_month_uid = feature_month_uid.where(feature_month_uid.notnull(), 0)
    return feature_month_uid

#只查询贷款贷款
def feature_month_uid_is_loan(month,prefix_name):
    feature_month_uid_is_loan = feature_month_uid(month,prefix_name)
    feature_month_uid_is_loan = feature_month_uid_is_loan[feature_month_uid_is_loan['is_loan'+prefix_name] == 1.0]
    feature_month_uid_is_loan['loan_sum'+prefix_name] = feature_month_uid_is_loan['loan_sum'+prefix_name]
    feature_month_uid_is_loan['limit'+prefix_name] = feature_month_uid_is_loan['limit'+prefix_name]
    feature_month_uid_is_loan['loan_sum_mean'+prefix_name] = feature_month_uid_is_loan['loan_sum_mean'+prefix_name]
    return feature_month_uid_is_loan

# 取各月数据
#feature_month_uid_8 = feature_month_uid('2016-08','_8')
#feature_month_uid_9 = feature_month_uid('2016-09','_9')
#feature_month_uid_10 = feature_month_uid('2016-10','_10')
#feature_month_uid_11 = feature_month_uid('2016-11','_11')

feature_month_uid_is_loan_8 = feature_month_uid_is_loan('2016-08','_8')
feature_month_uid_is_loan_9 = feature_month_uid_is_loan('2016-09','_9')
feature_month_uid_is_loan_10 = feature_month_uid_is_loan('2016-10','_10')
feature_month_uid_is_loan_11 = feature_month_uid_is_loan('2016-11','_11')

# 合并每个月数据
#feature_month_uid_8_9 = pd.merge(feature_month_uid_8,feature_month_uid_9,how='outer',on='uid')
#feature_month_uid_10_11 = pd.merge(feature_month_uid_10,feature_month_uid_11,how='outer',on='uid')
#feature_month_uid_all = pd.merge(feature_month_uid_8_9,feature_month_uid_10_11,how='outer',on='uid')

feature_month_uid_is_loan_8_9 = pd.merge(feature_month_uid_is_loan_8,feature_month_uid_is_loan_9,how='outer',on='uid')
feature_month_uid_is_loan_10_11 = pd.merge(feature_month_uid_is_loan_10,feature_month_uid_is_loan_11,how='outer',on='uid')
feature_month_uid_is_loan_all = pd.merge(feature_month_uid_is_loan_8_9,feature_month_uid_is_loan_10_11,how='outer',on='uid')


# 替换所有nan为0
#feature_month_uid_all_replace_none = feature_month_uid_all.where(feature_month_uid_all.notnull(), 0)
feature_month_uid_is_loan_all_replace_none = feature_month_uid_is_loan_all.where(feature_month_uid_is_loan_all.notnull(), 0)

# 按uid从小到大排序
#feature_month_uid_all_replace_none = feature_month_uid_all_replace_none.sort_values(by='uid')
feature_month_uid_is_loan_all_replace_none = feature_month_uid_is_loan_all_replace_none.sort_values(by='uid')

In [2]:
feature_month_uid_is_loan_all_replace_none

,uid,click_count_8,price_by_max_qty_8,qty_max_8,discount_by_max_qty_8,age_8,limit_8,loan_count_8,loan_sum_8,loan_sum_mean_8,...,price_by_max_qty_11,qty_max_11,discount_by_max_qty_11,age_11,limit_11,loan_count_11,loan_sum_11,loan_sum_mean_11,is_loan_11,diff_11
0,4,50.0,14.665733,4.0,11.159862,30.0,5.292154,2.0,10.445692,5.222846,...,2.866304,1.0,0.000000,30.0,5.292154,1.0,4.723017,4.723017,1.0,0.569137
33190,5,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,...,60.407578,48.0,21.030112,30.0,6.292055,1.0,6.584075,6.584075,1.0,-0.292020
19534,6,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,...,27.724517,43.0,13.759507,25.0,6.292055,1.0,6.266691,6.266691,1.0,0.025363
33191,17,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,...,0.000000,0.0,0.000000,30.0,5.292154,1.0,5.657311,5.657311,1.0,-0.365157
19535,19,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,...,0.000000,0.0,0.000000,50.0,5.974677,1.0,5.885770,5.885770,1.0,0.088907
1,20,29.0,18.710273,8.0,16.992488,35.0,6.292055,6.0,30.573341,5.095557,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000
2,23,79.0,25.752178,9.0,15.675134,35.0,5.722768,5.0,19.312973,3.862595,...,3.296981,2.0,0.000000,35.0,5.722768,1.0,3.862595,3.862595,1.0,1.860174
27941,26,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000
19536,27,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000
3,28,33.0,9.793121,3.0,6.589657,25.0,6.153414,1.0,4.292651,4.292651,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000


In [3]:
feature_month_uid_is_loan_all_replace_none.columns

Index(['uid', 'click_count_8', 'price_by_max_qty_8', 'qty_max_8',
       'discount_by_max_qty_8', 'age_8', 'limit_8', 'loan_count_8',
       'loan_sum_8', 'loan_sum_mean_8', 'is_loan_8', 'diff_8', 'click_count_9',
       'price_by_max_qty_9', 'qty_max_9', 'discount_by_max_qty_9', 'age_9',
       'limit_9', 'loan_count_9', 'loan_sum_9', 'loan_sum_mean_9', 'is_loan_9',
       'diff_9', 'click_count_10', 'price_by_max_qty_10', 'qty_max_10',
       'discount_by_max_qty_10', 'age_10', 'limit_10', 'loan_count_10',
       'loan_sum_10', 'loan_sum_mean_10', 'is_loan_10', 'diff_10',
       'click_count_11', 'price_by_max_qty_11', 'qty_max_11',
       'discount_by_max_qty_11', 'age_11', 'limit_11', 'loan_count_11',
       'loan_sum_11', 'loan_sum_mean_11', 'is_loan_11', 'diff_11'],
      dtype='object')

In [4]:
#9,10,11月份都没有借贷的人，直接删除，判断其12月的总贷款额为0
loan_none = feature_month_uid_is_loan_all_replace_none[(feature_month_uid_is_loan_all_replace_none['loan_sum_mean_9']==0.000000) & (feature_month_uid_is_loan_all_replace_none['loan_sum_mean_10']==0.000000) & (feature_month_uid_is_loan_all_replace_none['loan_sum_mean_11']==0.000000)]
for i in loan_none['uid']:
    feature_month_uid_is_loan_all_replace_none.drop(feature_month_uid_is_loan_all_replace_none.index[feature_month_uid_is_loan_all_replace_none['uid']==i],inplace=True)
del feature_month_uid_is_loan_all_replace_none['age_9']
del feature_month_uid_is_loan_all_replace_none['age_10']
del feature_month_uid_is_loan_all_replace_none['age_11']
del feature_month_uid_is_loan_all_replace_none['limit_9']
del feature_month_uid_is_loan_all_replace_none['limit_10']
del feature_month_uid_is_loan_all_replace_none['limit_11']
feature_month_uid_is_loan_all_replace_none = feature_month_uid_is_loan_all_replace_none.reset_index()
feature_month_uid_is_loan_all_replace_none
del feature_month_uid_is_loan_all_replace_none['index']

In [6]:
del feature_month_uid_is_loan_all_replace_none['age_8']

In [8]:
feature_month_uid_is_loan_all_replace_none = pd.merge(feature_month_uid_is_loan_all_replace_none,file_user,how = 'inner',on='uid')

In [10]:
del feature_month_uid_is_loan_all_replace_none['active_date']

In [11]:
import numpy as np
from sklearn.svm import SVR
import matplotlib.pyplot as plt

In [13]:
train = feature_month_uid_is_loan_all_replace_none
X = train.values
Y = train['loan_sum_mean_8'].values
svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)
y_rbf = svr_rbf.fit(X, Y).predict(X)

In [14]:
y_rbf

array([ 5.12266154,  0.10010973,  0.10010923, ...,  5.83154709,
        4.43216765,  6.52927114])

In [15]:
feature_month_uid_is_loan_all_replace_none['svm_8'] = y_rbf

In [16]:
train = feature_month_uid_is_loan_all_replace_none
X = train.values
Y = train['loan_sum_mean_9'].values
svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)
y_rbf9 = svr_rbf.fit(X, Y).predict(X)

In [18]:
feature_month_uid_is_loan_all_replace_none['svm_9'] = y_rbf9

In [19]:
train = feature_month_uid_is_loan_all_replace_none
X = train.values
Y = train['loan_sum_mean_10'].values
svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.2)
y_rbf10 = svr_rbf.fit(X, Y).predict(X)

In [20]:
feature_month_uid_is_loan_all_replace_none['svm_10'] = y_rbf10

In [21]:
train = feature_month_uid_is_loan_all_replace_none
X = train.values
Y = train['loan_sum_mean_11'].values
svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.2)
y_rbf11 = svr_rbf.fit(X, Y).predict(X)

In [22]:
y_rbf11

array([ 4.62315055,  6.48400783,  6.16718637, ...,  5.8317599 ,
        4.64250294,  6.52941166])

In [23]:
SVM_rbf = pd.DataFrame({'uid':train['uid'],'SVM_rbf':y_rbf11})
SVM_rbf = SVM_rbf[['uid','SVM_rbf']]
SVM_rbf.to_csv('SVM_rbf.csv')

In [25]:
from sklearn.tree import DecisionTreeRegressor
regr_1 = DecisionTreeRegressor(max_depth=2)
regr_2 = DecisionTreeRegressor(max_depth=5)
regr_1.fit(X, Y)
regr_2.fit(X, Y)

DecisionTreeRegressor(criterion='mse', max_depth=5, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [26]:
y_1 = regr_1.predict(X)
y_2 = regr_2.predict(X)

In [28]:
DT = pd.DataFrame({'uid':train['uid'],'DT1':y_1,'DT2':y_2})
DT = DT[['uid','DT1','DT2']]
DT.to_csv('DT.csv')

In [35]:
import xgboost as xgb
Y = train['loan_sum_mean_11'].values
train = xgb.DMatrix(feature_month_uid_is_loan_all_replace_none,label = Y)

# 设置参数
param = {'max_depth': 5,
         'eta': 1,
         'silent': 0,
         'objective': 'reg:logistic'}

num_round = 5

# 训练模型
bst = xgb.train(param, train, num_round)

preds = bst.predict(train)

TypeError: 'DMatrix' object is not subscriptable